In [1]:
# -*- coding: utf-8 -*-
"""
Generated by ArcGIS ModelBuilder on : 2024-01-22 16:18:29
"""
import arcpy
from arcpy.ia import *
from sys import argv

In [2]:
# GDB_C = r'C:\\Users\\wb527163\\GIS\\Mali_Cdrive\\Mali_Health_FY24\\Mali_Health_FY24_C\\Mali_Health_FY24_C.gdb\\'
# GDB_temp = r'C:\\Users\\wb527163\\GIS\\Mali_Cdrive\\Mali_Health_FY24\\Mali_Health_FY24_C\\NF_pop.gdb\\'

# GDB_B = r'C:\\Users\\wb527163\\GIS\\Mali_Cdrive\\Mali_Health_FY24\\Mali_Health_FY24_B.gdb\\'
# GDB_A = r'C:\\Users\\wb527163\\GIS\\Mali_Cdrive\\Mali_Health_FY24\\Mali_Health_FY24.gdb\\'
# GDB_R = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\'


GDB_python = r'D:\\Mali_FY24_D\\Mali_Health_FY24_python.gdb\\'
GDB_temp = r'D:\\Mali_FY24_D\\NF_scratch.gdb\\'
GDB_ACA = r'D:\\Mali_FY24_D\\NF_ACA1.gdb\\'
GDB_RCA = r'R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA1.gdb\\'

In [3]:
arcpy.env.workspace  = GDB_RCA[:-2]
rasters = arcpy.ListRasters("*", "")
print(rasters)
# arcpy.env.workspace = GDB_python[:-2]
# arcpy.env.scratchWorkspace = GDB_temp[:-2]
# print(arcpy.env.scratchGDB)

['NF_1_ACA', 'NF_2_ACA', 'NF_3_ACA', 'NF_4_ACA', 'NF_5_ACA', 'NF_6_ACA', 'NF_7_ACA', 'NF_8_ACA', 'NF_9_ACA', 'NF_10_ACA', 'NF_11_ACA', 'NF_12_ACA', 'NF_13_ACA', 'NF_14_ACA', 'NF_15_ACA', 'NF_16_ACA', 'NF_17_ACA', 'NF_18_ACA', 'NF_19_ACA', 'NF_20_ACA', 'NF_21_ACA', 'NF_22_ACA', 'NF_23_ACA', 'NF_24_ACA', 'NF_25_ACA', 'NF_26_ACA', 'NF_27_ACA', 'NF_28_ACA', 'NF_29_ACA', 'NF_30_ACA', 'NF_31_ACA', 'NF_32_ACA', 'NF_33_ACA', 'NF_34_ACA', 'NF_35_ACA']


In [4]:
rast = rasters[0]
print('"' + "{}".format(rast) + '"')

"NF_1_ACA"


In [11]:
def PeopleGainNewFacility(RCA_Rasters = GDB_RCA[:-2], ACA_masks = GDB_ACA[:-2]):  # Population stats for HF plus 1 NF

    # To allow overwriting outputs change overwriteOutput option to True.
    arcpy.env.overwriteOutput = True

    # Check out any necessary licenses.
    arcpy.CheckOutExtension("spatial")
    arcpy.CheckOutExtension("ImageAnalyst")
    arcpy.CheckOutExtension("ImageExt")
    
    
    # Inputs
    Eligible_NF = GDB_python + "Eligible_NF"
    ADM1_Dissolve = GDB_python + "ADM1_Dissolve"
    wpop = arcpy.Raster(GDB_python + "wpop")
    pxHFj_RCA_Full = arcpy.Raster(GDB_python + "pxHFj_RCA_Full")
    COMREF_MXmax = arcpy.Raster(GDB_python + "COMREF_MXmax")
    
    # Intermediate outputs
    pNF_ACA_tb = GDB_temp + "pNF_ACA_tb"
    pxHFj1_RCA_tb = GDB_temp + "pxHFj1_RCA_tb"
    pxHFj_RCA_tb = GDB_temp + "pxHFj_RCA_tb"
    pG20j1_tb = GDB_temp + "pG20j1_tb"
    
    # Final storage of values
    NF2 = Eligible_NF + "_2"
    
    arcpy.env.workspace  = RCA_Rasters
#     rasters = arcpy.ListRasters("*", "")
    rasters = ['NF_30_ACA', 'NF_31_ACA', 'NF_32_ACA', 'NF_33_ACA', 'NF_34_ACA', 'NF_35_ACA']
    print('Iterating through: ', rasters)

    for raster in rasters: 
        
        ACA_mask = ACA_masks + "\\" + raster
        RCA_selection = RCA_Rasters + "\\" + raster
        expr = '"' + "{}".format(raster) + '"'
        print('\n'.join([ACA_mask, RCA_selection, expr]))

        # Process: Select Layer By Attribute (Select Layer By Attribute) (management)
        Eligible_NF_selection, Count = arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=Eligible_NF, 
            where_clause=f"NF_code = '{raster}' And pNF_ACA IS NULL")

        # Process: If Value Is (If Value Is) ()
        if Count is "0":
            print('Already calculated for ' + raster + '.')
        else:
            print('\n\n---Running WorldPop within NF_ACA catchment for %s. %s---' % (raster, time.ctime()))

            
# # People in the New Facility catchment area

            # Process: RCalc_PplWithin6h (Raster Calculator) (ia)
            #RCalc_PplWithin6h = pNF_ACA
#             with arcpy.EnvManager(mask=ACA_mask):
#                 pNF_ACA = arcpy.ia.RasterCalculator(wpop)
#                 #pNF_ACA.save(RCalc_PplWithin6h)
                
            with arcpy.EnvManager(cellSize=wpop):
                wpop_mask = arcpy.sa.ExtractByMask(wpop, ACA_mask)#; 
#                 wpop_mask.save(pNF_ACA)
                
            print("Raster calculator finished. %s" % time.ctime())


            # Process: Zstat_PplWithin6h (Zonal Statistics as Table) (ia)
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, 
                                                zone_field="OBJECTID", 
                                                in_value_raster=wpop_mask, 
                                                out_table=pNF_ACA_tb, 
                                                ignore_nodata="DATA", 
                                                statistics_type="SUM", 
                                                process_as_multidimensional="CURRENT_SLICE", 
                                                percentile_values=90, 
                                                percentile_interpolation_type="AUTO_DETECT")#.save(zStat1)
            
            print("Zonal stats finished. %s" % time.ctime())
            
            # Process: Calculate Field (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=pNF_ACA_tb, 
                                            field="NF_code", 
                                            expression=expr, 
                                            expression_type="PYTHON3", 
                                            field_type="TEXT")#[0]
                
            #arcpy.management.AlterField(pxHFj1_RCA_tb, "SUM", "SUM1")
                
            # Process: Join Field (3) (Join Field) (management)
#             arcpy.management.JoinField(in_data=NF2, 
#                                        in_field="NF_code", 
#                                        join_table=pNF_ACA_tb, 
#                                        join_field="NF_code", 
#                                        fields=["SUM"])#[0]

            arcpy.management.JoinField(NF2, 
                                       "NF_code", 
                                       pNF_ACA_tb, 
                                       "NF_code", 
                                       "SUM")
    
            print("Joined to copy of New Facilities. %s" % time.ctime())

            # Process: Calculate Field (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=NF2, 
                                            field="pNF_ACA", 
                                            expression="!SUM! if !SUM! is not None else !pNF_ACA!", 
                                            expression_type="PYTHON3")#[0]
            
            print("Transferred to permanent attribute table. %s" % time.ctime())

            # Process: Delete Field (Delete Field) (management)
            arcpy.management.DeleteField(in_table=NF2, 
                                         drop_field=["SUM"], 
                                         method="DELETE_FIELDS")#[0]

            
            print("Deleted temp field. %s" % time.ctime())
            
# # People-minutes in the New Facility catchment area
        # Process: Select Layer By Attribute (Select Layer By Attribute) (management)
        Eligible_NF_selection, Count = arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=Eligible_NF, 
            where_clause=f"NF_code = '{raster}' And pxHFj1_RCA IS NULL")

        # Process: If Value Is (If Value Is) ()
        if Count is "0":
            print('Already calculated for ' + raster + '.')
        else:
            print('\n\n---Running People-minutes with a New Facility for %s. %s---' % (raster, time.ctime()))

            
            multiplied = Con(~IsNull(Raster(ACA_mask)), 
                                Raster(wpop) * Raster(RCA_selection)) 
            
#             print("Calculated. %s" % time.ctime())
#             output_raster.save(r"D:\Mali_FY24_D\NF_scratch.gdb\con_isnullra_23_v4")

            print("Raster calculator finished. %s" % time.ctime())


            # Process: Zonal Statistics as Table (Zonal Statistics as Table) (ia)
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, 
                                                zone_field="OBJECTID", 
                                                in_value_raster=multiplied, 
                                                out_table=pxHFj1_RCA_tb, 
                                                ignore_nodata="DATA", 
                                                statistics_type="SUM", 
                                                process_as_multidimensional="CURRENT_SLICE", 
                                                percentile_values=90, 
                                                percentile_interpolation_type="AUTO_DETECT")
                #.save(Zonal_Statistics_as_Table)

            print("Zonal stats finished. %s" % time.ctime())
            
            #arcpy.management.AlterField(pxHFj1_RCA_tb, 'SUM', 'SUM2')

            # Process: Calculate Field (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=pxHFj1_RCA_tb, 
                                            field="NF_code", 
                                            expression=expr, 
                                            expression_type="PYTHON3",
                                            field_type="TEXT")#[0]

            # Process: Join Field (Join Field) (management)
#             arcpy.management.JoinField(in_data=NF2, 
#                                        in_field="NF_code", 
#                                        join_table=pxHFj1_RCA_tb, 
#                                        join_field="OBJECTID_1", 
#                                        fields=["SUM"])#[0]
            
            arcpy.management.JoinField(NF2, 
                                       "NF_code", 
                                       pxHFj1_RCA_tb, #r"D:\Mali_FY24_D\NF_scratch.gdb\pxHFj1_RCA_tb", 
                                       "NF_code", 
                                       "SUM")
            
            print("Joined to copy of New Facilities. %s" % time.ctime())

            # Process: Calculate Field (4) (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=NF2, 
                                            field="pxHFj1_RCA", 
                                            expression="!SUM! if !SUM! is not None else !pxHFj1_RCA!", 
                                            expression_type="PYTHON3")#[0]
            
            print("Transferred to permanent attribute table. %s" % time.ctime())

            # Process: Delete Field (4) (Delete Field) (management)
            arcpy.management.DeleteField(in_table=NF2, 
                                         drop_field=["SUM"], 
                                         method="DELETE_FIELDS")#[0]
            
            print("Deleted temp field. %s" % time.ctime())

# # People-minutes in the New Facility catchment area before introducing New Facility
        # Process: Select Layer By Attribute (Select Layer By Attribute) (management)
        Eligible_NF_selection, Count = arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=Eligible_NF, 
            where_clause=f"NF_code = '{raster}' And pxHFj_RCA IS NULL")

        # Process: If Value Is (If Value Is) ()
        if Count is "0":
            print('Already calculated for ' + raster + '.')
        else:
            print('\n\n---Running People-minutes at baseline for %s. %s---' % (raster, time.ctime()))

            # Process: RCalc_PplMin_HF (Raster Calculator) (ia)
            #RCalc_PplMin_HF = pxHFj_ACA
#             with arcpy.EnvManager(mask= ACA_mask):
#             pxHFj_mask = Con(~IsNull(Raster(ACA_mask)), Raster(pxHFj_RCA_Full))
                #pxHFj_ACA.save(RCalc_PplMin_HF)
            with arcpy.EnvManager(cellSize=pxHFj_RCA_Full):
                pxHFj_mask = arcpy.sa.ExtractByMask(pxHFj_RCA_Full, ACA_mask)#; 
            print("Raster calculator finished. %s" % time.ctime())


            # Process: Zonal Statistics as Table (4) (Zonal Statistics as Table) (ia)
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, 
                                                zone_field="OBJECTID", 
                                                in_value_raster=pxHFj_mask, 
                                                out_table=pxHFj_RCA_tb, 
                                                ignore_nodata="DATA", 
                                                statistics_type="SUM", 
                                                process_as_multidimensional="CURRENT_SLICE", 
                                                percentile_values=90, 
                                                percentile_interpolation_type="AUTO_DETECT")
                #.save(Zonal_Statistics_as_Table_4_)
            
            print("Zonal stats finished. %s" % time.ctime())
            
            # Process: Calculate Field (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=pxHFj_RCA_tb, 
                                            field="NF_code", 
                                            expression=expr, 
                                            expression_type="PYTHON3",
                                            field_type="TEXT")#[0]

            #arcpy.management.AlterField(pxHFj_RCA_tb, 'SUM', 'SUM3')

            # Process: Join Field (4) (Join Field) (management)
#             arcpy.management.JoinField(in_data=NF2, 
#                                        in_field="NF_code", 
#                                        join_table=pxHFj_RCA_tb, 
#                                        join_field="OBJECTID_1", 
#                                        fields=["SUM"])#[0]
            
            arcpy.management.JoinField(NF2, 
                                       "NF_code", 
                                       pxHFj_RCA_tb, #r"D:\Mali_FY24_D\NF_scratch.gdb\pxHFj_RCA_tb", 
                                       "NF_code", 
                                       "SUM")
            
            print("Joined to copy of New Facilities. %s" % time.ctime())

            # Process: Calculate Field (3) (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=NF2, 
                                            field="pxHFj_RCA", 
                                            expression="!SUM! if !SUM! is not None else !pxHFj_RCA!", 
                                            expression_type="PYTHON3")#[0]
            
            print("Transferred to permanent attribute table. %s" % time.ctime())

            # Process: Delete Field (3) (Delete Field) (management)
            arcpy.management.DeleteField(in_table=NF2, 
                                         drop_field=["SUM"], 
                                         method="DELETE_FIELDS")#[0]
            
            print("Deleted temp field. %s" % time.ctime())

# # People with reduction in travel time (20%+ gain in minutes saved).
        # Process: Select Layer By Attribute (Select Layer By Attribute) (management)
        Eligible_NF_selection, Count = arcpy.management.SelectLayerByAttribute(
            in_layer_or_view=Eligible_NF, 
            where_clause=f"NF_code = '{raster}' And pG20j1 IS NULL")

        # Process: If Value Is (If Value Is) ()
        if Count is "0":
            print('Already calculated for ' + raster + '.')
        else:
            print('\n\n---Running People whose travel time is reduced for %s. %s---' % (raster, time.ctime()))

            # Process: RCalc_PplGain20 (Raster Calculator) (ia)
            #RCalc_PplGain20 = pG20j1
            with arcpy.EnvManager(mask= ACA_mask):
                pG20j1 = Con( ((COMREF_MXmax - RCA_selection)/COMREF_MXmax) >= 0.2,  wpop)
                #pG20j1.save(RCalc_PplGain20)
            
            print("Raster calculator finished. %s" % time.ctime())

            # Process: Zonal Statistics as Table (2) (Zonal Statistics as Table) (ia)
            with arcpy.EnvManager(parallelProcessingFactor="80%"):
                arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, 
                                                zone_field="OBJECTID", 
                                                in_value_raster=pG20j1, 
                                                out_table=pG20j1_tb, 
                                                ignore_nodata="DATA", 
                                                statistics_type="SUM", 
                                                process_as_multidimensional="CURRENT_SLICE", 
                                                percentile_values=90, 
                                                percentile_interpolation_type="AUTO_DETECT")
                #.save(Zonal_Statistics_as_Table_2_)
            
            print("Zonal stats finished. %s" % time.ctime())
            
            # Process: Calculate Field (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=pG20j1_tb, 
                                            field="NF_code", 
                                            expression=expr, 
                                            expression_type="PYTHON3", 
                                            field_type="TEXT")#[0]
            
            #arcpy.management.AlterField(pG20j1_tb, 'SUM', 'SUM4')
            
            # Process: Join Field (2) (Join Field) (management)
#             arcpy.management.JoinField(in_data=NF2, 
#                                        in_field="NF_code", 
#                                        join_table=pG20j1_tb, 
#                                        join_field="OBJECTID_1", 
#                                        fields=["SUM"])#[0]
            
            arcpy.management.JoinField(NF2, 
                                       "NF_code", 
                                       pG20j1, 
                                       "NF_code", 
                                       "SUM")
            
            print("Joined to copy of New Facilities. %s" % time.ctime())

            # Process: Calculate Field (2) (Calculate Field) (management)
            arcpy.management.CalculateField(in_table=NF2, 
                                            field="pG20j1", 
                                            expression="!SUM! if !SUM! is not None else !pG20j1!", 
                                            expression_type="PYTHON3")#[0]
            
            print("Transferred to permanent attribute table. %s" % time.ctime())

            # Process: Delete Field (2) (Delete Field) (management)
            arcpy.management.DeleteField(in_table=NF2, 
                                         drop_field=["SUM"], 
                                         method="DELETE_FIELDS")#[0]
            
            print("Deleted temp field. %s" % time.ctime())

In [12]:
if __name__ == '__main__':
    # Global Environment settings
    with arcpy.EnvManager(scratchWorkspace=GDB_temp[:-2], workspace=GDB_python[:-2]):
        PeopleGainNewFacility(*argv[1:])

Iterating through:  ['NF_30_ACA', 'NF_31_ACA', 'NF_32_ACA', 'NF_33_ACA', 'NF_34_ACA', 'NF_35_ACA']
Already calculated for NF_30_ACA.
Already calculated for NF_30_ACA.
Already calculated for NF_30_ACA.
Already calculated for NF_31_ACA.
Already calculated for NF_31_ACA.
Already calculated for NF_31_ACA.
Already calculated for NF_32_ACA.
Already calculated for NF_32_ACA.
Already calculated for NF_32_ACA.


---Running WorldPop within NF_ACA catchment for NF_33_ACA. Wed Jan 24 11:26:33 2024---
Raster calculator finished. Wed Jan 24 11:27:29 2024
Zonal stats finished. Wed Jan 24 11:27:38 2024
Joined to copy of New Facilities. Wed Jan 24 11:27:39 2024
Transferred to permanent attribute table. Wed Jan 24 11:27:40 2024
Deleted temp field. Wed Jan 24 11:27:40 2024


---Running People-minutes with a New Facility for NF_33_ACA. Wed Jan 24 11:27:43 2024---
Raster calculator finished. Wed Jan 24 11:27:44 2024
Zonal stats finished. Wed Jan 24 11:39:32 2024
Joined to copy of New Facilities. Wed Jan 24

In [5]:
RCA_Rasters = GDB_RCA[:-2]
ACA_masks = GDB_ACA[:-2]

arcpy.env.overwriteOutput = True

# Check out any necessary licenses.
arcpy.CheckOutExtension("spatial")
arcpy.CheckOutExtension("ImageAnalyst")
arcpy.CheckOutExtension("ImageExt")

''

In [6]:
# Inputs
Eligible_NF = GDB_python + "Eligible_NF"
ADM1_Dissolve = GDB_python + "ADM1_Dissolve"
wpop = arcpy.Raster(GDB_python + "wpop")
pxHFj_RCA_Full = arcpy.Raster(GDB_python + "pxHFj_RCA_Full")
COMREF_MXmax = arcpy.Raster(GDB_python + "COMREF_MXmax")

# Intermediate outputs
pNF_ACA_tb = GDB_temp + "pNF_ACA_tb"
pxHFj1_RCA_tb = GDB_temp + "pxHFj1_RCA_tb"
pxHFj_RCA_tb = GDB_temp + "pxHFj_RCA_tb"
pG20j1_tb = GDB_temp + "pG20j1_tb"

In [7]:
# Final storage of values
NF2 = Eligible_NF + "_3"

arcpy.env.workspace  = RCA_Rasters
raster = 'NF_23_ACA'

In [8]:
ACA_mask = ACA_masks + "\\" + raster
RCA_selection = RCA_Rasters + "\\" + raster
expr = '"' + "{}".format(raster) + '"'
print('\n'.join([ACA_mask, RCA_selection, expr]))

D:\\Mali_FY24_D\\NF_ACA1.gdb\NF_23_ACA
R:\\MLI\\GEO\\Team\\Projects\\MLI_Health_FY24_R\\NF_RCA1.gdb\NF_23_ACA
"NF_23_ACA"


In [9]:
# multiplied = Con(~IsNull(ACA_mask), wpop * RCA_selection)

# print("Raster calculator finished. %s" % time.ctime())

Raster calculator finished. Thu Jan 25 11:48:54 2024


In [17]:
output_raster = Con(~IsNull(Raster(r"D:\Mali_FY24_D\NF_ACA1.gdb\NF_23_ACA")), 
                    Raster(r"D:\Mali_FY24_D\Mali_Health_FY24_python.gdb\wpop") * Raster(r"R:/MLI/GEO/Team/Projects/MLI_Health_FY24_R/NF_RCA1.gdb/NF_23_ACA")) 
print("Calculated. %s" % time.ctime())

output_raster.save(r"D:\Mali_FY24_D\NF_scratch.gdb\con_isnullra_23_v4")

print("Raster calculator finished. %s" % time.ctime())

Raster calculator finished. Thu Jan 25 12:13:25 2024


In [18]:
with arcpy.EnvManager(parallelProcessingFactor="80%"):
    arcpy.ia.ZonalStatisticsAsTable(in_zone_data=ADM1_Dissolve, 
                                    zone_field="OBJECTID", 
                                    in_value_raster=output_raster, 
                                    out_table=pxHFj1_RCA_tb, 
                                    ignore_nodata="DATA", 
                                    statistics_type="SUM", 
                                    process_as_multidimensional="CURRENT_SLICE", 
                                    percentile_values=90, 
                                    percentile_interpolation_type="AUTO_DETECT")

print("Zonal stats finished. %s" % time.ctime())

Zonal stats finished. Thu Jan 25 12:14:42 2024


In [19]:
arcpy.management.CalculateField(in_table=pxHFj1_RCA_tb, 
                                field="NF_code", 
                                expression=expr, 
                                expression_type="PYTHON3",
                                field_type="TEXT")#[0]

<Result 'D:\\\\Mali_FY24_D\\\\NF_scratch.gdb\\\\pxHFj1_RCA_tb'>

In [20]:
arcpy.management.JoinField(NF2, 
                           "NF_code", 
                           pxHFj1_RCA_tb,
                           "NF_code", 
                           "SUM")

print("Joined to copy of New Facilities. %s" % time.ctime())

Joined to copy of New Facilities. Thu Jan 25 12:16:33 2024


In [21]:
arcpy.management.CalculateField(in_table=NF2, 
                                field="pxHFj1_RCA", 
                                expression="!SUM! if !SUM! is not None else !pxHFj1_RCA!", 
                                expression_type="PYTHON3")

print("Transferred to permanent attribute table. %s" % time.ctime())

Transferred to permanent attribute table. Thu Jan 25 12:16:39 2024


In [22]:
arcpy.management.DeleteField(in_table=NF2, 
                             drop_field=["SUM"], 
                             method="DELETE_FIELDS")

print("Deleted temp field. %s" % time.ctime())

Deleted temp field. Thu Jan 25 12:16:42 2024
